In [ ]:
!pip install xgboost

In [ ]:
import os
import pandas as pd

dfs = []
for dirname, _, filenames in os.walk(RUNWAY_DATA_PATH):
    for filename in filenames:
        if filename.endswith(".csv"):
            d = pd.read_csv(os.path.join(dirname, filename))
        elif filename.endswith(".parquet"):
            d = pd.read_parquet(os.path.join(dirname, filename))
        else:
            raise ValueError("Not valid file type")
        dfs += [d]

df = pd.concat(dfs)
df.columns = df.columns.map(lambda x: x.lower())

In [ ]:
X_columns = [
    "activepower",
    "ambienttemperatue",
    "bearingshafttemperature",
    "blade1pitchangle",
    "blade2pitchangle",
    "blade3pitchangle",
    "controlboxtemperature",
    "gearboxbearingtemperature",
    "gearboxoiltemperature",
    "generatorrpm",
    "generatorwinding1temperature",
    "generatorwinding2temperature",
    "hubtemperature",
    "mainboxtemperature",
    "nacelleposition",
    "reactivepower",
    "rotorrpm",
    "turbinestatus",
    "winddirection",
    "windspeed",
]
y_column = "activepower"


X_df = df[X_columns]
y_df = df[y_column]

In [ ]:
from sklearn.model_selection import train_test_split

## Split data into training and testing sets
X_train, X_valid, y_train, y_valid = train_test_split(X_df, y_df, test_size=0.2)

In [ ]:
xgb.XGBRegressor?

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error


params = {
    "objective": "reg:squarederror",
    "learning_rate": LEARNING_RATE,
    "max_depth": MAX_DEPTH,
    "alpha": ALPHA,
    "n_estimators": N_ESTIMATORS,
    }

regr = xgb.XGBRegressor(
    objective=params["objective"],
    learning_rate=params["learning_rate"],
    max_depth=params["max_depth"],
    alpha=params["alpha"],
    n_estimators=params["n_estimators"],
)

In [ ]:
regr.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])

y_pred = regr.predict(X_valid)
mae = mean_absolute_error(y_pred, y_valid)
mse = mean_squared_error(y_pred, y_valid)

In [ ]:
import pandas as pd

class RunwayModel:
    def __init__(self, xgb_regressor):
        self._regr = xgb_regressor

    def predict(self, X):
        return pd.DataFrame(
            {
                "activepower": self._regr.predict(X),
            }
        )

In [ ]:
runway_model = RunwayModel(regr)

In [ ]:
input_sample = X_df.sample(1)
input_sample

In [ ]:
import runway

runway.start_run()
runway.log_parameters(params)
runway.log_metric("valid_mae", mae)
runway.log_metric("valid_mse", mse)

runway.log_model(model_name="my-xgboost-regressor", model=runway_model, input_samples={"predict": input_sample})
